## Steps to build the Loan Eligibility model

1. Loading the dataset
2. Pre-processing the dataset
3. Building the Loan Prediction model

## 1. Loading the dataset

In [10]:
# importing libraries
import pandas as pd

# loading the dataset
train = pd.read_csv('loan_data.csv')
train.head()

,Gender,Married,ApplicantIncome,LoanAmount,Loan_Status
0,Male,Yes,4583,128000.0,N
1,Male,Yes,3000,66000.0,Y
2,Male,Yes,2583,120000.0,Y
3,Male,No,6000,141000.0,Y
4,Male,Yes,5417,267000.0,Y


## 2. Pre-processing the dataset

In [11]:
# converting categories into numbers
train['Gender']= train['Gender'].map({'Male':0, 'Female':1})
train['Married']= train['Married'].map({'No':0, 'Yes':1})
train['Loan_Status']= train['Loan_Status'].map({'N':0, 'Y':1})

In [12]:
train.head()

,Gender,Married,ApplicantIncome,LoanAmount,Loan_Status
0,0,1,4583,128000.0,0
1,0,1,3000,66000.0,1
2,0,1,2583,120000.0,1
3,0,0,6000,141000.0,1
4,0,1,5417,267000.0,1


In [21]:
# separating dependent and independent variables
X = train[['Gender', 'Married', 'ApplicantIncome', 'LoanAmount']]
y = train.Loan_Status

In [25]:
X.head()

,Gender,Married,ApplicantIncome,LoanAmount
0,0,1,4583,128000.0
1,0,1,3000,66000.0
2,0,1,2583,120000.0
3,0,0,6000,141000.0
4,0,1,5417,267000.0


In [26]:
y.head()

0    0
1    1
2    1
3    1
4    1
Name: Loan_Status, dtype: int64

## 3. Building the Loan Prediction model

In [27]:
# importing machine learning model
from sklearn.linear_model import LogisticRegression

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
max_features = 4
tfidf = TfidfVectorizer(max_features=max_features)#stop_words='english',)# norm = None)#)

In [29]:
# training the logistic regression model
X1 = tfidf.fit_transform(X)
model = LogisticRegression() 
model.fit(X1, y)

ValueError: Found input variables with inconsistent numbers of samples: [4, 577]

In [9]:
# saving the model 
import pickle 
##wb-write in binary mode 
pickle_out = open("classifier.pkl", mode = "wb") 
pickle.dump(model, pickle_out) 
pickle_out.close()

## Deploying the machine learning model using streamlit

1. Building the Frontend of the application
2. Loading and Pre-processing the data
3. Building the Machine Learning model to automate Loan Eligibility
4. Deploying the application

## 1. Building the Frontend of the application

1.1 Install Required Libraries<br>
1.2 Creating the Frontend of the app using Streamlit

### 1.1 Install Required Libraries##

In [10]:
# installing pyngrok
!pip install -q pyngrok

In [ ]:
# installing streamlit
!pip install -q streamlit

     |████████████████████████████████| 7.0MB 7.6MB/s 
     |████████████████████████████████| 112kB 56.1MB/s 
     |████████████████████████████████| 4.6MB 51.3MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
     |████████████████████████████████| 163kB 51.8MB/s 
     |████████████████████████████████| 122kB 53.7MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.


### 1.2. Creating the frontend of the app using streamlit

In [ ]:
# creating the script
%%writefile app.py

# importing required libraries
import pickle
import streamlit as st

# loading the trained model
pickle_in = open('classifier.pkl', 'rb') 
classifier = pickle.load(pickle_in)

# this is the main function in which we define our app  
def main():       
    # header of the page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Check your Loan Eligibility</h1> 
    </div> 
    """
    st.markdown(html_temp, unsafe_allow_html = True) 

    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female","Other"))
    Married = st.selectbox('Marital Status',("Unmarried","Married","Other")) 
    ApplicantIncome = st.number_input("Monthly Income in Rupees") 
    LoanAmount = st.number_input("Loan Amount in Rupees")
    result =""
      
    # when 'Check' is clicked, make the prediction and store it 
    if st.button("Check"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount) 
        st.success('Your loan is {}'.format(result))
 
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, ApplicantIncome, LoanAmount): 

    # 2. Loading and Pre-processing the data 

    if Gender == "Male":
        Gender = 0
    else:
        Gender = 1

    if Married == "Married":
        Married = 1
    else:
        Married = 0

    # 3. Building the model to automate Loan Eligibility 

    # if (ApplicantIncome >= 50000):
    #     loan_status = 'Approved'
    # elif (LoanAmount < 500000):
    #     loan_status = 'Approved'
    # else:
    #     loan_status = 'Rejected'
    # return loan_status

    prediction = classifier.predict( 
        [[Gender, Married, ApplicantIncome, LoanAmount]])
     
    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred
     
if __name__=='__main__': 
    main()

Writing app.py


## 4. Deploying the application

In [1]:
# running the app
!streamlit run MLapp.py

^C


In [2]:
!ngrok config add-authtoken 22mgHT1EwrXH9EBP0nyaHLJJ7xE_SFP2HiTjbZPZP8g7jgbn

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [3]:
# making the locally-hosted web application to be publicly accessible
from pyngrok import ngrok

public_url = ngrok.connect('8509')
public_url

t=2022-06-12T01:34:48+0800 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


<NgrokTunnel: "http://1cf4-2406-3003-206f-3fb8-3841-e7a6-6c28-990a.ngrok.io" -> "http://localhost:8509">